Input data

In [ ]:
# Load the shapefile of the study area (it should be in the assets in GEE, just copy and past its path)
polygon = ee.FeatureCollection('users/ahmed_mohsen250/Shp_Danube_in_Hungary') 

# Define the time range
start_date = '2020-01-01'
end_date = '2020-12-31'
# the colud cover is set to 10%, if you need to change it, adjsut it in the code below

Processing code

In [ ]:
import ee

# Initialize the Earth Engine library
ee.Initialize()  

# Filter the image collection for your area and time range
image_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterBounds(polygon)
                    .filterDate(start_date, end_date)
                    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10))  # Adjust cloud cover percentage if needed

# Check the number of available images
image_count = image_collection.size().getInfo()
print(f'Number of available images: {image_count}')

# Step 1: Add a date property to each image
def add_date(image):
    return image.set('date', image.date().format('YYYY-MM-dd'))

image_collection_with_date = image_collection.map(add_date)

# Step 2: Create a list of unique dates
dates = image_collection_with_date.distinct('date').aggregate_array('date')

# Step 3: Create a dictionary to store images for each date and create mosaics
def create_mosaics_for_dates(date):
    # Filter images for the specific date
    images_for_date = image_collection_with_date.filter(ee.Filter.eq('date', date))
    # Create a mosaic for the date
    return images_for_date.mosaic().set('date', date)

# Step 4: Create a list of mosaics for each date
mosaics = dates.map(create_mosaics_for_dates)

# Print the number of mosaicked images created
mosaic_count = mosaics.size().getInfo()
print(f'Number of mosaicked images created: {mosaic_count}')

# Step 5: Cloud, cirrus cloud, shadow, and snow masking function
def mask_clouds(image):
    scl = image.select('SCL')  # Scene Classification Layer
    # Mask cloud (value 3), shadow (value 8), cirrus (value 10), and snow (value 11)
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(mask)

# Step 6: Function to calculate NDWI
def calculate_ndwi(mosaic):
    # Apply cloud mask
    masked_mosaic = mask_clouds(mosaic)
    
    # Ensure the input is an ee.Image
    green_band = masked_mosaic.select('B3')  # Green band
    nir_band = masked_mosaic.select('B8')    # Near-infrared band
    ndwi = green_band.subtract(nir_band).divide(green_band.add(nir_band)).rename('NDWI')
    
    return ndwi.set('system:time_start', masked_mosaic.get('system:time_start')).set('date', mosaic.get('date'))

# Convert mosaics from ee.List to ee.ImageCollection
mosaic_images = ee.ImageCollection.fromImages(mosaics)

# Step 7: Calculate NDWI for each mosaicked image
ndwi_images = mosaic_images.map(calculate_ndwi)

# Step 8: Function to check if the mosaic covers more than 90% of the polygon
def is_full_coverage(ndwi_image):
    polygon_area = polygon.geometry().area().getInfo()  # Total area of the polygon in square meters

    # Calculate the area of valid (non-masked) pixels
    pixel_area_image = ee.Image.pixelArea().mask(ndwi_image.mask())  # Image of pixel areas, masked by valid pixels
    valid_area = pixel_area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon.geometry(),
        scale=10,  # Match the scale of the image
        maxPixels=1e13
    ).get('area')

    if valid_area is None:
        return False  # No valid area at all
    
    valid_area = ee.Number(valid_area).getInfo()
    
    # Check if the valid area is greater than 90% of the polygon's area
    return valid_area / polygon_area > 0.90

# Step 9: Export NDWI images that cover more than 90% of the polygon
def export_ndwi_image(ndwi_image):
    date = ndwi_image.get('date').getInfo()  # Get the date for naming
    if is_full_coverage(ndwi_image):  # Check if the image covers more than 90% of the polygon
        export_task = ee.batch.Export.image.toDrive(
            image=ndwi_image.clip(polygon),  # Clip to the shapefile area
            description=f'NDWI_{date}',      # Description for the export task
            folder='GEE_Exports_2',            # Google Drive folder to save images
            fileNamePrefix=f'NDWI_{date}',   # Prefix for the filename
            region=polygon.geometry(),       # Define the export region
            scale=10,                        # Scale in meters per pixel
            maxPixels=1e13                   # Maximum number of pixels to export
        )
        export_task.start()  # Start the export task
        print(f"Exporting NDWI image for {date}")
    else:
        print(f"NDWI image for {date} does not cover more than 90% of the polygon and will not be exported.")

# Retrieve the list of NDWI images
ndwi_images_list = ndwi_images.toList(ndwi_images.size())

# Export each NDWI image that meets the coverage criteria
for i in range(ndwi_images.size().getInfo()):
    ndwi_image = ee.Image(ndwi_images_list.get(i))
    export_ndwi_image(ndwi_image)

print("Export tasks started for NDWI images that cover more than 90% of the polygon.")
